In [1]:
## Daniel Marten 
# Create table with TPM Counts

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import statistics
import sys
import os
import statsmodels.api as sm
import statsmodels.stats.weightstats as sm_stats
import seaborn as sns
import scipy
import scipy.stats as scistats

from statsmodels.formula.api import ols as formula_OLS

from cmapPy.pandasGEXpress.parse_gct import parse as tpm_parser




Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Read in table with Phylogenetic information 
phylo_df = pd.read_csv(r'gs://ug-wphu/gtex_analysis/victor_2149+Ens89/Hs_Ens89+2149_PS_seq_etc_hg38.txt',sep='\t')
phylo_original = phylo_df.copy() # making a copy in case we want to look at it later on
phylo_df.rename(columns={'Name':'Gene_ID'},inplace=True)
phylo_df.set_index('Protein_ID',inplace=True)
phylo_df.index.names = ['ENSP']
phylo_df.sort_index(inplace=True)
updated_mapping = phylo_df.copy()

,PS,Gene_ID,Description,Plength,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence
ENSP,,,,,,,,,,,,,
Baz_Hs_1,31,NaN,ENST00000308604.5_18272_457,52,Not_Gap_Gene,chr2,NaN,NaN,-,111429413,111429572,MTDTENHDSSPSSTSTCCPPITAGMQLKDSLGPGSNCPLWTLRPLH...,ATGACAGACACTGAAAATCACGACTCATCCCCCTCCAGCACCTCTA...
Baz_Hs_10,31,NaN,ENST00000411630.2_23991_594,59,Not_Gap_Gene,chr4,NaN,NaN,+,52713673,52713853,MLVATGQCSRCFMFTFSTFSFNCHNSEVDSVRDRLPQDHSAPANSM...,ATGCTGGTGGCAACAGGGCAGTGTAGCAGGTGCTTCATGTTCACCT...
Baz_Hs_103,31,NaN,ENST00000499346.2_27384_333,84,Not_Gap_Gene,chr5,NaN,NaN,-,128082767,128083022,MLGAFRSGPQPLPEPRARCVPQPGLLWALTRRRESPLVTPGLNLEE...,ATGCTGGGGGCTTTCCGGTCGGGGCCGCAGCCGCTTCCGGAGCCGC...
Baz_Hs_108,24,NaN,ENST00000501177.3_12701_390,84,Not_Gap_Gene,chr16,NaN,NaN,-,54919086,54925136,MLAEIHPKAGLQSLQFIMELLYWLLEGGDSEDKEDATGNVEMKNIQ...,ATGTTGGCTGAAATTCATCCCAAGGCTGGTCTGCAAAGTCTGCAAT...
Baz_Hs_112,25,NaN,ENST00000503704.1_24073_293,52,Not_Gap_Gene,chr4,NaN,NaN,-,82900166,82900435,MRSREAGPKLRRIQEPANGSPGAVSETGGYREERLSDAEIMGKLLA...,ATGCGAAGCAGAGAGGCAGGACCAAAATTGAGGCGAATCCAGGAAC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
vdp2013_S4_994,31,NaN,HF584391,69,Not_Gap_Gene,chrX,NaN,NaN,-,143628143,143628353,MLYTHNTEFNLKRQICFVPQCKTFVSLCFVKQTQENWYTCTSWVLY...,ATGCTTTATACACATAATACTGAATTTAACCTCAAGAGGCAAATCT...
vdp2013_S4_995,31,NaN,HF583960,46,Not_Gap_Gene,chr10,NaN,NaN,+,104020518,104021665,MREWLSIRNMRIKCEIFSCSVKPMSANCISCRMKNATCWLSMRLRN,ATGAGAGAATGGCTCAGCATCAGAAACATGAGAATCAAATGCGAGA...
vdp2013_S4_997,31,NaN,HF548108,40,Not_Gap_Gene,chr6,NaN,NaN,+,70860637,70860760,MFAYKGSSYHVSNTSNSINPTPKLASNPVGRYCMIKCLII,ATGTTTGCATATAAGGGAAGTAGTTATCATGTTAGTAATACCTCTA...


In [3]:
# Gene Median TPM Counts in GRCh38 from GTEx v8 
naive_38 = pd.read_csv(r'/Users/marten/Downloads/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct',sep='\t',skiprows=2)
naive_38

,Name,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,ENSG00000223972.5,DDX11L1,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.166403,0.000000,0.000000,0.000000,0.00000
1,ENSG00000227232.5,WASH7P,4.064030,3.371110,2.68549,4.04762,3.90076,3.639630,5.16375,1.43859,...,5.932980,6.13265,4.193780,5.926310,3.062480,4.702530,6.272550,7.190010,5.745540,2.64743
2,ENSG00000278267.1,MIR6859-1,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,ENSG00000243485.5,MIR1302-2HG,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.054223,0.000000,0.000000,0.000000,0.00000
4,ENSG00000237613.2,FAM138A,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56195,ENSG00000198695.2,MT-ND6,3517.220000,3598.090000,6637.79000,6374.49000,4972.73000,4635.000000,8275.20000,9631.77000,...,8919.460000,8113.22000,9764.490000,5477.360000,9628.750000,1676.590000,2520.410000,3644.000000,2237.430000,960.29100
56196,ENSG00000210194.1,MT-TE,11.903800,11.203400,24.41390,22.65940,20.58800,18.257600,27.50560,40.58860,...,32.932100,31.33400,35.456000,24.934900,33.496300,2.871770,5.795670,10.370100,5.724510,3.49736
56197,ENSG00000198727.2,MT-CYB,23839.100000,25511.700000,28799.40000,12919.40000,15943.20000,13051.500000,23663.20000,32931.80000,...,19058.200000,17223.10000,24521.300000,11578.600000,27193.100000,15337.800000,19868.500000,15025.000000,12803.400000,2305.49000
56198,ENSG00000210195.2,MT-TT,0.616205,0.697191,1.50928,0.00000,0.00000,0.544659,0.00000,3.44476,...,0.884701,1.11364,0.815455,0.783492,0.803858,0.443781,0.000000,0.000000,0.000000,0.00000


In [4]:
# 54592 unique ENSGs !!
len(naive_38.Description.unique())

54592

In [5]:
# mart_export_hg38_20230907
# From biomart Ens93, find mappings for ENSG<->ENST<->ENSP

translationDict = pd.read_csv(r'/Users/marten/Downloads/biomart_export_grch38_ens93_jul2018_20231121.txt',sep='\t')

display(translationDict)

translationDict['Ensembl_gene_length'] = translationDict['Gene end (bp)'] - translationDict['Gene start (bp)']

translationDict.rename(columns={'Gene stable ID':'ENSG','Transcript stable ID':'ENST','Protein stable ID':'ENSP'},inplace=True)

translationDict = translationDict[['ENSG','ENST','ENSP','Ensembl_gene_length']]

transEnsg = translationDict.copy()
transEnsg.set_index('ENSG',inplace=True)

transEnst = translationDict.copy()
transEnst.set_index('ENST',inplace=True)

transEnsp = translationDict.copy()
transEnsp.set_index('ENSP',inplace=True)

# display(transEnsg,transEnst,transEnsp) # uncomment to display

,Gene stable ID,Transcript stable ID,Protein stable ID,Gene start (bp),Gene end (bp)
0,ENSG00000276626,ENST00000612820,NaN,124792319,124792562
1,ENSG00000201317,ENST00000364447,NaN,92700819,92700934
2,ENSG00000200823,ENST00000363953,NaN,100951856,100951933
3,ENSG00000221598,ENST00000408671,NaN,45200954,45201019
4,ENSG00000199595,ENST00000362725,NaN,161699506,161699607
...,...,...,...,...,...
224538,ENSG00000285363,ENST00000642800,NaN,129728744,129730445
224539,ENSG00000285114,ENST00000645112,NaN,129757225,129795938
224540,ENSG00000285114,ENST00000642712,ENSP00000496548,129757225,129795938
224541,ENSG00000285114,ENST00000646090,ENSP00000494855,129757225,129795938


,ENST,ENSP,Ensembl_gene_length
ENSG,,,
ENSG00000276626,ENST00000612820,NaN,243
ENSG00000201317,ENST00000364447,NaN,115
ENSG00000200823,ENST00000363953,NaN,77
ENSG00000221598,ENST00000408671,NaN,65
ENSG00000199595,ENST00000362725,NaN,101
...,...,...,...
ENSG00000285363,ENST00000642800,NaN,1701
ENSG00000285114,ENST00000645112,NaN,38713
ENSG00000285114,ENST00000642712,ENSP00000496548,38713


,ENSG,ENSP,Ensembl_gene_length
ENST,,,
ENST00000612820,ENSG00000276626,NaN,243
ENST00000364447,ENSG00000201317,NaN,115
ENST00000363953,ENSG00000200823,NaN,77
ENST00000408671,ENSG00000221598,NaN,65
ENST00000362725,ENSG00000199595,NaN,101
...,...,...,...
ENST00000642800,ENSG00000285363,NaN,1701
ENST00000645112,ENSG00000285114,NaN,38713
ENST00000642712,ENSG00000285114,ENSP00000496548,38713


,ENSG,ENST,Ensembl_gene_length
ENSP,,,
NaN,ENSG00000276626,ENST00000612820,243
NaN,ENSG00000201317,ENST00000364447,115
NaN,ENSG00000200823,ENST00000363953,77
NaN,ENSG00000221598,ENST00000408671,65
NaN,ENSG00000199595,ENST00000362725,101
...,...,...,...
NaN,ENSG00000285363,ENST00000642800,1701
NaN,ENSG00000285114,ENST00000645112,38713
ENSP00000496548,ENSG00000285114,ENST00000642712,38713


In [6]:
# Calculating number of ENSPs
ensps = ['ENSP' in xi for xi in updated_mapping.index]
sum(ensps) # there are 19334 ENSP normal annotated proteins

19334

In [7]:
# Split ENSGs to without information after decimal
naive_38['Name'] = [nameio.split('.')[0] for nameio in naive_38.Name]

In [8]:
# Filter phylo mapping to ONLY genes with 'ensp'
updated_mapping_ensp = updated_mapping[updated_mapping.index.str.contains('ENSP')]

In [9]:
# Join gtex_grch38 information with ENSG mapping dictionary
# set index to the mapped-to ENSP, then use that to 
# get information from the phylogenetic dictionary 
gtex_merged = (naive_38.set_index('Name')).join(transEnsg)
gtex_merged['ENSG'] = gtex_merged.index
gtex_merged = gtex_merged.set_index('ENSP')
gtex_full = gtex_merged.join(updated_mapping,rsuffix='_2')

In [10]:
# Dropping genes without Phylogenetic information would give us 19,029 genes/proteins
gtex_full.dropna(subset='PS')

,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),...,Plength,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence
ENSP,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,ARF5,155.033000,130.395000,104.215000,173.917000,167.389000,169.550000,144.037000,93.188000,132.819000,...,180.0,Not_Gap_Gene,chr7,127588344.0,127591705.0,+,127588498.0,127591299.0,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...
ENSP00000000412,M6PR,54.179700,46.653400,49.364500,53.353600,51.712000,56.593000,62.392600,11.360100,13.774700,...,277.0,Not_Gap_Gene,chr12,8940364.0,8949955.0,-,8941817.0,8946404.0,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...
ENSP00000000442,ESRRA,34.075500,33.017900,34.561200,24.257400,27.962400,26.555800,46.109100,13.058700,17.573900,...,423.0,Not_Gap_Gene,chr11,64305577.0,64316738.0,+,64307179.0,64315966.0,MSSQVVGIEPLYIKAEPASPDSPKGSSETETEPPVALAPGPAPTRC...,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...
ENSP00000001008,FKBP4,24.843200,34.593100,34.090000,40.999300,40.903100,52.456600,50.941200,22.969000,32.425800,...,459.0,Not_Gap_Gene,chr12,2794952.0,2805423.0,+,2795139.0,2803258.0,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...
ENSP00000001146,CYP26B1,35.635300,26.576700,3.870610,10.873600,9.130600,5.292380,4.654000,4.021700,4.125860,...,512.0,Not_Gap_Gene,chr2,72129237.0,72148038.0,-,72132226.0,72147834.0,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSP00000492767,TRIM43B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,446.0,Not_Gap_Gene,chr2,95477163.0,95482714.0,-,95477163.0,95482714.0,MDSDFSHAFQKELTCVICLNYLVDPVTICCGHSFCRPCLCLSWEEA...,ATGGACTCAGACTTCTCACATGCCTTCCAGAAGGAACTCACCTGTG...
ENSP00000492773,FAM217A,0.137665,0.074539,0.092826,0.063639,0.055228,0.068421,0.080345,0.011690,0.018658,...,642.0,Not_Gap_Gene,chr6,4068695.0,4087027.0,-,4068695.0,4087027.0,MPRLTQRVAVSPEYLLLPPRHGGCDKISWLPEGKVFSNVHREKISS...,ATGCCCAGATTAACACAGAGGGTTGCTGTGTCTCCTGAATACCTTC...
ENSP00000492787,RP11-321E2.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,198.0,Not_Gap_Gene,chr5,17518366.0,17518963.0,+,17518366.0,17518963.0,METGRQTGVSAEMLAMPRGLKGSKKDGIPEDLDGNLEAPRDQEGEL...,ATGGAGACCGGCAGGCAAACAGGTGTGTCTGCTGAGATGCTCGCCA...


In [11]:
# Create a list of ENSPs that are not present - 322 long
not_present = []
for xi in updated_mapping.index:
    if xi not in gtex_full.index:
        if 'ENSP' in xi:
            not_present.append(xi)

In [12]:
len(not_present)

322

In [13]:
np = updated_mapping.loc[not_present]
np

,PS,Gene_ID,Description,Plength,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence
ENSP,,,,,,,,,,,,,
ENSP00000222388,1,ABCF2,ATP binding cassette subfamily F member 2 [So...,634,Not_Gap_Gene,chr7,151207836.0,151227166.0,-,151208070,151226458,MPSDLAKKKAAKKKEAAKARQRPRKGHEENGDVVTEPQVAEKNEAN...,ATGCCCTCCGACCTGGCCAAGAAGAAGGCAGCCAAAAAGAAGGAGG...
ENSP00000246421,2,SLC35E2,solute carrier family 35 member E2 [Source:HG...,266,Not_Gap_Gene,chr1,1732241.0,1745992.0,-,1732454,1739557,MSSSVKTPALEELVPGSEEKPKGRSPLSWGSLFGHRSEKIVFAKSD...,ATGTCGTCCTCGGTGAAAACCCCAGCACTGGAAGAGCTGGTTCCTG...
ENSP00000247005,4,GDF1,growth differentiation factor 1 [Source:HGNC ...,372,Not_Gap_Gene,chr19,18868544.0,18896096.0,-,18868596,18870307,MPPPQQGPCGHHLLLLLALLLPSLPLTRAPVPPGPAAALLQALGLR...,ATGCCACCGCCGCAGCAAGGTCCCTGCGGCCACCACCTCCTCCTCC...
ENSP00000250937,1,DOHH,deoxyhypusine hydroxylase [Source:HGNC Symbol...,302,Not_Gap_Gene,chr19,3490820.0,3500637.0,-,3491491,3496814,MVTEQEVDAIGQTLVDPKQPLQARFRALFTLRGLGGPGAIAWISQA...,ATGGTGACGGAGCAGGAGGTGGATGCCATCGGGCAGACGCTGGTGG...
ENSP00000251337,2,GNAT2,G protein subunit alpha transducin 2 [Source:...,354,Not_Gap_Gene,chr1,109603266.0,109613057.0,-,109603353,109612870,MGSGASAEDKELAKRSKELEKKLQEDADKEAKTVKLLLLGAGESGK...,ATGGGAAGTGGAGCCAGTGCTGAGGACAAAGAACTGGCCAAGAGGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSP00000492441,1,RPS10-NUDT3,RPS10-NUDT3 readthrough [Source:HGNC Symbol;A...,291,Not_Gap_Gene,chr6,34284886.0,34426071.0,-,34288752,34425221,MLMPKKNRIAIYELLFKEGVMVAKKDVHMPKHPELADKNVPNLHVM...,ATGTTGATGCCTAAGAAGAACCGGATTGCCATTTATGAACTCCTTT...
ENSP00000492446,2,NaN,AC007326.4,180,Not_Gap_Gene,chr22,18906237.0,18947732.0,+,18906374,18939702,MERYAGALEEVADGARQQERHYQLLSALQSLVKELPSSFQQRLSYT...,ATGGAGCGCTACGCGGGCGCCTTGGAGGAGGTGGCGGACGGTGCCC...
ENSP00000492745,1,NaN,AC118553.2,713,Not_Gap_Gene,chr1,99970017.0,100082494.0,+,99993554,100082209,MFANLKYVSLGILVFQTTSLVLTMRYSRTLKEEGPRYLSSTAVVVA...,ATGTTCGCCAACCTAAAATACGTTTCCCTGGGAATTTTGGTCTTTC...


In [14]:
# Output ENSPs not present in TPM analysis
np.to_csv('ENSP_not_in_TPM_analysis_grch38_202301121.csv')

In [15]:
# Output only of ENSPs that we have full information for 
gtex_full_undropped = gtex_full.copy()
gtex_full.dropna(subset='PS',inplace=True)
gtex_full

,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),...,Plength,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence
ENSP,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,ARF5,155.033000,130.395000,104.215000,173.917000,167.389000,169.550000,144.037000,93.188000,132.819000,...,180.0,Not_Gap_Gene,chr7,127588344.0,127591705.0,+,127588498.0,127591299.0,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...
ENSP00000000412,M6PR,54.179700,46.653400,49.364500,53.353600,51.712000,56.593000,62.392600,11.360100,13.774700,...,277.0,Not_Gap_Gene,chr12,8940364.0,8949955.0,-,8941817.0,8946404.0,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...
ENSP00000000442,ESRRA,34.075500,33.017900,34.561200,24.257400,27.962400,26.555800,46.109100,13.058700,17.573900,...,423.0,Not_Gap_Gene,chr11,64305577.0,64316738.0,+,64307179.0,64315966.0,MSSQVVGIEPLYIKAEPASPDSPKGSSETETEPPVALAPGPAPTRC...,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...
ENSP00000001008,FKBP4,24.843200,34.593100,34.090000,40.999300,40.903100,52.456600,50.941200,22.969000,32.425800,...,459.0,Not_Gap_Gene,chr12,2794952.0,2805423.0,+,2795139.0,2803258.0,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...
ENSP00000001146,CYP26B1,35.635300,26.576700,3.870610,10.873600,9.130600,5.292380,4.654000,4.021700,4.125860,...,512.0,Not_Gap_Gene,chr2,72129237.0,72148038.0,-,72132226.0,72147834.0,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSP00000492767,TRIM43B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,446.0,Not_Gap_Gene,chr2,95477163.0,95482714.0,-,95477163.0,95482714.0,MDSDFSHAFQKELTCVICLNYLVDPVTICCGHSFCRPCLCLSWEEA...,ATGGACTCAGACTTCTCACATGCCTTCCAGAAGGAACTCACCTGTG...
ENSP00000492773,FAM217A,0.137665,0.074539,0.092826,0.063639,0.055228,0.068421,0.080345,0.011690,0.018658,...,642.0,Not_Gap_Gene,chr6,4068695.0,4087027.0,-,4068695.0,4087027.0,MPRLTQRVAVSPEYLLLPPRHGGCDKISWLPEGKVFSNVHREKISS...,ATGCCCAGATTAACACAGAGGGTTGCTGTGTCTCCTGAATACCTTC...
ENSP00000492787,RP11-321E2.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,198.0,Not_Gap_Gene,chr5,17518366.0,17518963.0,+,17518366.0,17518963.0,METGRQTGVSAEMLAMPRGLKGSKKDGIPEDLDGNLEAPRDQEGEL...,ATGGAGACCGGCAGGCAAACAGGTGTGTCTGCTGAGATGCTCGCCA...


In [16]:
# Add ages and eras using a dictionary 

def rev_dict(input_dict):
    ret_dict = {}
    for key,val in input_dict.items():
        for subval in val: 
            ret_dict[subval] = key
            
    return ret_dict
    

five_era = {
    '1-Ancient':list(range(0,4)),
    '2-Metazoa':list(range(4,8)),
    '3-Chordate':list(range(8,18)),
    '4-Mammal':list(range(18,23)),
    '5-Primate':list(range(23,32)),
}

five_rev_era = rev_dict(five_era)
        
gtex_full['PS'] = gtex_full['PS'].astype('int')
gtex_full['Era-5'] = [five_rev_era[psHere] for psHere in gtex_full['PS']]

display(gtex_full[['PS','Era-5']])


,PS,Era-5
ENSP,,
ENSP00000000233,1,1-Ancient
ENSP00000000412,2,1-Ancient
ENSP00000000442,1,1-Ancient
ENSP00000001008,1,1-Ancient
ENSP00000001146,1,1-Ancient
...,...,...
ENSP00000492767,2,1-Ancient
ENSP00000492773,8,3-Chordate
ENSP00000492787,1,1-Ancient


In [17]:
# output for use in plotting and significance calculations
gtex_full.to_csv('marten_gtex_withera_forsigwork_grch38.tsv',sep='\t')
gtex_full

,Description,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),...,Gap_Gene?,Chr,Gene_Start_hg38,Gene_End_hg38,Strand,CDS_Start_hg38,CDS_End_hg38,Protein_Sequence,CDS_Sequence,Era-5
ENSP,,,,,,,,,,,,,,,,,,,,,
ENSP00000000233,ARF5,155.033000,130.395000,104.215000,173.917000,167.389000,169.550000,144.037000,93.188000,132.819000,...,Not_Gap_Gene,chr7,127588344.0,127591705.0,+,127588498.0,127591299.0,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...,1-Ancient
ENSP00000000412,M6PR,54.179700,46.653400,49.364500,53.353600,51.712000,56.593000,62.392600,11.360100,13.774700,...,Not_Gap_Gene,chr12,8940364.0,8949955.0,-,8941817.0,8946404.0,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,1-Ancient
ENSP00000000442,ESRRA,34.075500,33.017900,34.561200,24.257400,27.962400,26.555800,46.109100,13.058700,17.573900,...,Not_Gap_Gene,chr11,64305577.0,64316738.0,+,64307179.0,64315966.0,MSSQVVGIEPLYIKAEPASPDSPKGSSETETEPPVALAPGPAPTRC...,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...,1-Ancient
ENSP00000001008,FKBP4,24.843200,34.593100,34.090000,40.999300,40.903100,52.456600,50.941200,22.969000,32.425800,...,Not_Gap_Gene,chr12,2794952.0,2805423.0,+,2795139.0,2803258.0,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,1-Ancient
ENSP00000001146,CYP26B1,35.635300,26.576700,3.870610,10.873600,9.130600,5.292380,4.654000,4.021700,4.125860,...,Not_Gap_Gene,chr2,72129237.0,72148038.0,-,72132226.0,72147834.0,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,1-Ancient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSP00000492767,TRIM43B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Not_Gap_Gene,chr2,95477163.0,95482714.0,-,95477163.0,95482714.0,MDSDFSHAFQKELTCVICLNYLVDPVTICCGHSFCRPCLCLSWEEA...,ATGGACTCAGACTTCTCACATGCCTTCCAGAAGGAACTCACCTGTG...,1-Ancient
ENSP00000492773,FAM217A,0.137665,0.074539,0.092826,0.063639,0.055228,0.068421,0.080345,0.011690,0.018658,...,Not_Gap_Gene,chr6,4068695.0,4087027.0,-,4068695.0,4087027.0,MPRLTQRVAVSPEYLLLPPRHGGCDKISWLPEGKVFSNVHREKISS...,ATGCCCAGATTAACACAGAGGGTTGCTGTGTCTCCTGAATACCTTC...,3-Chordate
ENSP00000492787,RP11-321E2.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Not_Gap_Gene,chr5,17518366.0,17518963.0,+,17518366.0,17518963.0,METGRQTGVSAEMLAMPRGLKGSKKDGIPEDLDGNLEAPRDQEGEL...,ATGGAGACCGGCAGGCAAACAGGTGTGTCTGCTGAGATGCTCGCCA...,1-Ancient


In [18]:
# Create dictionary of tissue categories
ectoDerm = [
    "Nerve_Tibial",
    "Skin Not Sun Exposed (Suprapubic)",
    "Skin_Sun_Exposed_Lower_leg",
    "Breast - Mammary Tissue",
    "Pituitary",
    "Adrenal Gland",
    "Brain - Cerebellum",
    "Brain - Cortex",
    "Cervix - Endocervix",
    "Minor Salivary Gland",
    ]

brainDerm = [
    "Brain - Nucleus accumbens (basal ganglia)",
    "Brain - Caudate (basal ganglia)",
    "Brain - Cerebellar Hemisphere",
    "Brain - Frontal Cortex (BA9)",
    "Brain - Hypothalamus",
    "Brain - Putamen (basal ganglia)",
    "Brain - Hippocampus",
    "Brain - Anterior cingulate cortex (BA24)",
    "Brain - Amygdala",
    "Brain - Spinal cord (cervical c-1)",
    "Brain - Substantia nigra",
    "Brain - Cerebellum",
    "Brain - Cortex",]

mesoDerm = [
    "Muscle - Skeletal",
    "Whole Blood",
    "Adipose - Subcutaneous",
    "Cells - Cultured fibroblasts",
    "Adipose - Visceral (Omentum)",
    "Esophagus - Muscularis",
    "Artery - Aorta",
    "Heart - Left Ventricle",
    "Heart - Atrial Appendage",
    "Spleen",
    "Prostate",
    "Artery - Coronary",
    "Artery - Tibial",
    "Cells - EBV-transformed lymphocytes",
    "Vagina",
    "Uterus",
    "Kidney - Cortex",
    "Bladder",
    "Cervix - Ectocervix",
    "Fallopian Tube",
    "Kidney - Medulla",]

endoDerm = [
    "Thyroid",
    "Lung",
    "Esophagus - Mucosa",
    "Colon - Transverse",
    "Esophagus - Gastroesophageal Junction",
    "Stomach",
    "Colon - Sigmoid",
    "Pancreas",
    "Liver",
    "Small Intestine - Terminal Ileum"]

germline = [
    'Testis',
    'Ovary'
]

In [19]:
# perform some dictionary and renaming work
import re

dict_renaming = {}
allcolumns = gtex_full.columns

megaDerm = ectoDerm+mesoDerm+endoDerm+brainDerm+germline

for col in gtex_full.columns:
    dict_renaming[col] = re.sub(r'\W+', '', col).lower().replace('_','')

In [20]:
gtex_full.rename(columns=dict_renaming,inplace=True)

In [21]:
# dictionary of all layers to itself
germ_dict = {'ecto':[ectoDerm],
            'meso':[mesoDerm],
            'endo':[endoDerm],
            'brain':[brainDerm],
            'germline':[germline]}


# now each germ layer is index to a list of two lists
# 1 - list of names with capitals and formatting
# 2 - only alphanumerical lowercase fellas 
for dermkey in germ_dict.keys():
    germ_dict[dermkey].append([re.sub(r'\W+', '', xi).lower().replace('_','') for xi in germ_dict[dermkey][0]])

In [22]:
# Calculate the mean counts per layer for these groups 
for iterName,iterRow in gtex_full.iterrows():
    gtex_full.loc[iterName,'brain'] = iterRow[germ_dict['brain'][1]].mean()
    gtex_full.loc[iterName,'ecto'] = iterRow[germ_dict['ecto'][1]].mean()
    gtex_full.loc[iterName,'meso'] = iterRow[germ_dict['meso'][1]].mean()
    gtex_full.loc[iterName,'endo'] = iterRow[germ_dict['endo'][1]].mean()

In [23]:
# Output GTEx TPM data with lots of information - used for plotting and significance work 
gtex_full.to_csv('/Users/marten/Downloads/tpm_fulldata.tsv',sep='\t')

In [24]:
megaDerm_lower = [re.sub(r'\W+', '', xi).lower().replace('_','') for xi in sorted(list(set(megaDerm)))]

In [25]:
len(megaDerm_lower) # indeed 54 tissues only 

54

In [26]:
tpm_54 = gtex_full[megaDerm_lower+['enst', 'ensg', 'ps', 'geneid', 'description2', 'plength', 'gapgene',
       'chr', 'genestarthg38', 'geneendhg38', 'strand', 'cdsstarthg38',
       'cdsendhg38', 'proteinsequence', 'cdssequence','era5']]
tpm_54.to_csv('tpm_updated_54tissues_oldPS.tsv',sep='\t') # output TPMs with information for each tissue

In [27]:
# Set of tissues which are NOT germline 
nongerm_set = set(['adiposesubcutaneous', 'adiposevisceralomentum', 'adrenalgland',
       'arteryaorta', 'arterycoronary', 'arterytibial', 'bladder',
       'brainamygdala', 'brainanteriorcingulatecortexba24',
       'braincaudatebasalganglia', 'braincerebellarhemisphere',
       'braincerebellum', 'braincortex', 'brainfrontalcortexba9',
       'brainhippocampus', 'brainhypothalamus',
       'brainnucleusaccumbensbasalganglia', 'brainputamenbasalganglia',
       'brainspinalcordcervicalc1', 'brainsubstantianigra',
       'breastmammarytissue', 'cellsculturedfibroblasts',
       'cellsebvtransformedlymphocytes', 'cervixectocervix',
       'cervixendocervix', 'colonsigmoid', 'colontransverse',
       'esophagusgastroesophagealjunction', 'esophagusmucosa',
       'esophagusmuscularis', 'fallopiantube', 'heartatrialappendage',
       'heartleftventricle', 'kidneycortex', 'kidneymedulla', 'liver', 'lung',
       'minorsalivarygland', 'muscleskeletal', 'nervetibial',
       'pancreas', 'pituitary', 'prostate', 'skinnotsunexposedsuprapubic',
       'skinsunexposedlowerleg', 'smallintestineterminalileum', 'spleen',
       'stomach', 'thyroid', 'uterus', 'vagina', 'wholeblood'])

In [28]:
len(nongerm_set) # len is 54-2 = 52

52

In [29]:
# Output GTEx TPM information form Brain, Endo, Ecto, Meso, Testis, and Ovary 
tpm_6 = gtex_full[sorted(list(germ_dict.keys())[:-2])+['brain','testis','ovary','enst', 'ensg', 'ps', 'geneid', 'description2', 'plength', 'gapgene',
       'chr', 'genestarthg38', 'geneendhg38', 'strand', 'cdsstarthg38',
       'cdsendhg38', 'proteinsequence', 'cdssequence', 'era5']]
tpm_6.to_csv('marten_tpm_updated_6tissuegroups_oldPS.tsv',sep='\t') # output this, in 6 tissue categories
# including the two germ tissues 